In [1]:
from jaclang.jac.lexer import JacLexer
from jaclang.jac.parser import JacParser
from jaclang.jac.transform import TransformError

In [2]:
source = "/home/ubuntu/chandra/jaseci_labs/vscode-jaseci/server/tests/unit/test.jac"

In [13]:
import re

diagnostics = []
try:
    with open(source, "r") as f:
        source_str = f.read()
    lex = JacLexer(mod_path=source, input_ir=source_str).ir
    prse = JacParser(mod_path=source, input_ir=lex)
except TransformError as e:
    if e.errors_had:
        for err in e.errors_had:
            line = int(re.findall(r"Line (\d+)", err)[0].replace("Line ", ""))
            msg = err.split(",")[1].strip()
            diagnostics.append({
                "range": {
                    "start": {
                        "line": line - 1,
                        "character": 0
                    },
                    "end": {
                        "line": line - 1,
                        "character": 0
                    }
                },
                "message": msg,
                "severity": 'error'
            })
    if e.warnings_had:
        for err in e.warnings_had:
            line = int(re.findall(r"Line (\d+)", err)[0].replace("Line ", ""))
            msg = err.split(",")[1].strip()
            diagnostics.append({
                "range": {
                    "start": {
                        "line": line - 1,
                        "character": 0
                    },
                    "end": {
                        "line": line - 1,
                        "character": 0
                    }
                },
                "message": msg,
                "severity": 'warning'
            })

ERROR - Mod test.jac: Line 13, JParse Error, incorrect usage of "activities" (NAME)



['Mod test.jac: Line 13, JParse Error, incorrect usage of "activities" (NAME)\n']


In [14]:
diagnostics

[{'range': {'start': {'line': 12, 'character': 0},
   'end': {'line': 12, 'character': 0}},
  'message': 'JParse Error',
  'severity': 'error'}]